<a href="https://colab.research.google.com/github/chongzicbo/keras_tutorial/blob/master/tf.keras%E9%A2%84%E8%AE%AD%E7%BB%83%E6%A8%A1%E5%9E%8B%E4%BD%BF%E7%94%A8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

###tensorflow.keras预训练模型使用

In [1]:
from tensorflow.keras.applications.resnet50 import ResNet50
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.resnet50 import preprocess_input,decode_predictions
import numpy as np
import tensorflow as tf
from tensorflow import keras

In [0]:
tf.enable_eager_execution()

##使用Resnet50进行ImageNet分类

In [4]:
model=ResNet50(weights='imagenet')

102973440/102967424 [==============================] - 3s 0us/step


In [5]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
img_path="/content/drive/My Drive/data/d2l-zh-tensoflow/img/cat1.jpg"
img=image.load_img(img_path,target_size=(224,224))
x=image.img_to_array(img)
x=np.expand_dims(x,axis=0)
x=preprocess_input(x)

In [7]:
print(x.shape)

(1, 224, 224, 3)


In [8]:
preds=model.predict(x)
preds.shape

(1, 1000)

In [9]:
print('Predicted:',decode_predictions(preds,top=3)[0])

40960/35363 [==================================] - 0s 0us/step
Predicted: [('n02123045', 'tabby', 0.5392685), ('n02123159', 'tiger_cat', 0.3790904), ('n02124075', 'Egyptian_cat', 0.024531964)]


##使用VGG16提取特征

In [0]:
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.vgg16 import preprocess_input
import numpy as np

In [11]:
model=VGG16(weights='imagenet',include_top=False)
img=image.load_img(img_path,target_size=(224,224))
x=image.img_to_array(img)
x=np.expand_dims(x,axis=0)
x=preprocess_input(x)
features=model.predict(x)
features.shape

58892288/58889256 [==============================] - 2s 0us/step


(1, 7, 7, 512)

##从VGG19任意中间层抽取特征

In [0]:
from tensorflow.keras.applications.vgg19 import VGG19,preprocess_input
from tensorflow.keras.models import Model

In [0]:
base_model=VGG19(weights='imagenet')
model=Model(inputs=base_model.input,outputs=base_model.get_layer('block4_pool').output)
img=image.load_img(img_path,target_size=(224,224))
x=image.img_to_array(img)
x=np.expand_dims(x,axis=0)
x=preprocess_input(x)
block4_pool_features=model.predict(x)

In [8]:
block4_pool_features.shape

(1, 14, 14, 512)

##在新类上微调InceptionV3

In [0]:
from tensorflow.keras.applications.inception_v3 import InceptionV3
from tensorflow.keras.layers import Dense,GlobalAveragePooling2D
from tensorflow.keras.models import Model

In [0]:
base_model=InceptionV3(weights='imagenet',include_top=False) #构建不带分类器的预训练模型
#添加全局平均池化层
x=base_model.output
x=GlobalAveragePooling2D()(x)
#添加一个全连接层
x=Dense(1024,activation='relu')(x)
#添加一个分类器，假设我们有200个类
predictions=Dense(200,activation='softmax')(x)
#构建我们需要训练的完整模型
model=Model(inputs=base_model.input,outputs=predictions)
#首先，我们只训练顶部的几层（随机初始化的层）
#锁住所有Inception V3的卷积层
for layer in base_model.layers:
  layer.trainable=False

#编译模型，一定要在锁住层后操作
model.compile(optimizer='rmsprop',loss='categorical_crossentropy')
#在新的数据集上训练几代
model.fit_generator()

#顶层训练好后，开始微调InceptionV3的卷积层
#锁住底下的几层，然后训练其余的顶层
for i,layer in enumerate(base_model.layers):
  print(i,layer.name)

#训练最上面的两个Inception block
#锁住前面249层，然后放开之后的层
for layer in model.layers[:249]:
  layer.trainable=False
for layer in model.layers[249:]:
  layer.trainable=True
#需要重新编译模型，才能使上面的修改生效
from tensorflow.keras.optimizers import  SGD
model.compile(optimizer=SGD(lr=0.0001,momentum=0.9),loss='categorical_crossentropy')
#继续训练模型,这次训练最后两个Inception block和两个全连接层
model.fit_generator()